- 문서에서 토큰을 추출하기 위해선 str.split()과 같거나 그 이상의 기능을 수행하는 조작 메서드가 필요.
    - *we'll 과 같은 축약형을 we, will로 분리할 매커니즘이 필요.*
- 토큰을 모두 식별했다면 **어간 추출(stemming)**이 필요. **정규 표현식을 이용**해서 가능함.
- 다음으로 이 **토큰들을 이용해 단어 모음(bag of words)이라 부르는 문서의 벡터 표현을 생성.**
- ***단어 또는 토큰이 무엇을 표현할까?*** 문맥과 상황에 따라 다를 것.
    - don't를 하나의 의미 전달 단위로 볼 수도 있고, 2개의 단위로 볼 수도 있음.
    - ice cream같은 경우는 'ice cream'을 하나로 봐야할까, 'ice' + 'cream'으로 봐야 할까?
- 텍스트를 단어로 분할하기 위해선 어떤 매커니즘이 필요할까?
    - 한 단어로 구성된 **'Don't!'**를 보자.
    - 컴퓨터 측면에서 보면 이는 **단순히 1개의 단어**
    - 하지만 사람 측면에서 보면 **어떤 생략된 단어가 있음**을 알 수 있음.
        - 'Don't you dare!' 아니면 'Don't you do that!'을 줄인 것 일 것.
        - 이런 축약형 단어는 추후 chapter.4에서 살펴보자.
        - 지금은 우선 토큰들을 주어진대로 식별하는 토큰 생성기를 구축해보자.
- **n-gram** : ***문자열에서 추출된 n쌍의 토큰(단어벡터).***
    - 이를 이용하면 'ice cream'같은 단어를 'ice', 'cream', 'ice cream'으로 추출할 수 있음.
        - **벡터 표현이 1개가 아닌 2개(더 나아가 n개)로 이뤄지는 것.**
- 텍스트를 수치 벡터로 변환하는 것은 손실이 큰 추출과정.
    - But, 텍스트에 담긴 정보를 통해 ML모형을 구축할 수 있음. 어느정도의 정보는 유지하는 셈.

## Stemming Overview  <br>

- **어간 추출(stemming)**은 **한 단어의 여러 변형을 동일한 '통' or '군집'으로 묶는 것**을 의미.
- ***복수형을 단수형으로 바꾸기 위해 s를 그냥 지워도 될까?***
    - words에서 s를 지워 word를 얻을 수 있음.
    - bus에서 s를 지우면 bu. 이 방법이 통하지 않음.
    - 어떠한 매커니즘이 필요.
- **한 단어 또는 단어의 한 부분에 있는 개별 글자가 그 단어의 의미에 관한 정보를 제공함.**
- **글자 하나 때문에 단어의 의미가 완전히 달라질 수도 있음.**
- 전통적인 어간 추출 방식을 사용해보자.

## Vocabulary building with Token Generator

<br>

- **토큰화(Tokenization)** : 문서 분할(segmentation)의 한 종류. **주어진 텍스트를 더 작고 구체적인 정보를 담은 조각으로 쪼개는 것.**
- 텍스트를 토큰으로 나누는 분할을 Tokenization 또는 Token Generation이라 함.
- NLP의 기본 구축요소들에 **컴파일러 구성 요소를 다음과 같이 대응**할 수 있음.
    - Token Generator == Scanner, Lexer
    - Vocabulary == Vocabulary Dictionary (어휘집)
    - Parser == Compiler
    - Token, n-gram, etc... == Token, 기호
- **토큰화는 NLP 파이프라인 첫 단계**인 만큼 파이프라인 나머지 부분에 큰 영향을 미침.
    - **구조가 없는 자료인 자연어 텍스트를 정보 조각들로 분할.**
    - 이런 **정보 조각들을 개별 요소로 취급해 개수를 셀 수 있음.**
    - 이 **횟수를 그대로 벡터 성분으로 이용**해 벡터를 만들면 **문서를 대표하는 하나의 벡터 표현**이 됨.
        - 이렇게 생성된 **단어모음 벡터의 가장 일반적인 용도는 문서 검색.**

<br>

- 토큰화의 가장 간단한 방법은 **문자열 안 공백(whitespace)을 구분자(delimiter)로 사용**하는 것. **str.split()**
    - **한국어** 같은 경우는 **단어를 좀 더 세부적인 단위인 형태소로 분리하는 것이 나을 수도** 있음.
    - **명사들만 추출하고 조사들은 불용어로 처리**하는 것이 바람직한 경우들. (-이, -을 제거)


               split을 이용한 간단한 토큰 분할을 보자.
   

In [1]:
sentence = 'Charlie started Data Science study at the age of 23.'
sentence.split()

['Charlie',
 'started',
 'Data',
 'Science',
 'study',
 'at',
 'the',
 'age',
 'of',
 '23.']

- 문장 끝 23. 에 마침표가 포함되어 있는것만 빼면 적당히 나눠졌음.
    - **마침표 같은 문장 부호는 의미 있는 토큰과 분리하는 것이 원칙.**
        - 26. 이라는 토큰은 컴퓨터 언어에서 부동소수점 수 26.0을 나타내는덴 완벽.
        - But, 다른 단어 26들과는 구분되는 다른 토큰이 되어버림.
        - 26. 26! 26? 들의 문장 부호를 제거해 모두 26이 되도록 하는것이 좋을 것.
    - 좀 더 정교하게 한다면 이런 **문장부호들을 별개의 토큰으로 분리**할 수 있을 것.
- 문장부호 제거는 추후 진행하고, **각 단어를 나타내는 수치 벡터 표현**을 만들어보자.
    - 이를 **one-hot vector**라고 함.
    - **원핫 벡터들의 시퀀스는 기존의 원문 텍스트를 온전히 표현**할 수 있음.
    - 이 과정을 통해 **주어진 텍스트가 수치적인 자료구조(2차원 수치배열)로 변환**됨.
        - NLP의 첫번째 문제인 단어들을 수치들로 변환하는 문제가 여기서 해결되는 셈.

In [2]:
import numpy as np

token_sequence = sentence.split()

# 고유 단어들만을 모아 어휘사전을 만듦.
vocab = sorted(set(token_sequence))
', '.join(vocab) # 토큰을 숫자>영대문자>영소문자 순으로 정렬.

'23., Charlie, Data, Science, age, at, of, started, study, the'

In [3]:
num_tokens = len(token_sequence)
vocab_size = len(vocab)

# 너비가 어휘의 고유 단어 개수, 높이가 문서의 길이(토큰개수)인 빈 테이블(2차원 배열) 생성.
# 현재 예시에선 10*10 크기의 테이블이 생성됨.
onehot_vectors = np.zeros((num_tokens, vocab_size), int)

# 문장의 각 단어에 대해, 어휘에서 그 단어에 해당하는 위치의 테이블 값을 1로 설정.
for i, word in enumerate(token_sequence):
    onehot_vectors[i, vocab.index(word)] = 1

print(' '.join(vocab))
onehot_vectors

23. Charlie Data Science age at of started study the


array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

- 이 0과 1들이 문장의 원핫 단어 벡터들. 근데 뭐가 뭔지 알아보기가 힘듦.
- 그러니 pd.DataFrame으로 그려서 살펴보자.

In [4]:
import pandas as pd

pd.DataFrame(onehot_vectors, columns=vocab)

,23.,Charlie,Data,Science,age,at,of,started,study,the
0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0
9,1,0,0,0,0,0,0,0,0,0


- 원핫 벡터는 모든 성분 중 단 하나만 0이 아닌 대단히 sparse한 벡터.
    - 문장이 커지면 메모리 낭비가 매~우 심해짐.
- 좀더 보기 편하려면 **0 $\rightarrow$ ' '(공백)**으로 바꿔서 봐보자. 파이프라인 넣을때 이런 짓 하면 안됩니다.

In [5]:
df = pd.DataFrame(onehot_vectors, columns=vocab)
df[df == 0] = ' '
df

,23.,Charlie,Data,Science,age,at,of,started,study,the
0,,1,,,,,,,,
1,,,,,,,,1,,
2,,,1,,,,,,,
3,,,,1,,,,,,
4,,,,,,,,,1,
5,,,,,,1,,,,
6,,,,,,,,,,1
7,,,,,1,,,,,
8,,,,,,,1,,,
9,1,,,,,,,,,


- 문장 1개만 있는 문서의 이런 표현에서, **각 row는 한 단어에 대한 벡터.**
- 현재 문장은 10개의 단어, 각 단어는 한번씩만 사용 됐음.
    - **한 col의 1은 해당 어휘 단어가 문서의 해당 위치에 출현한다는 의미.**
    - **문서의 세번째 단어가 뭔지 알고싶으면 테이블의 3번째 row에서 값이 1인 col**을 찾으면 됨.
        - 3번째 단어를 알고싶으니 **row index==2 에서 value가 1인 col을 찾으면 Data.**
        - 실제 **sentence를 보면 Data가 3번째에 출현**함.
        - NLP 파이프라인에서 문장의 한 단어 'Data'는 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]라는 벡터가 되는 셈.
- 단어의 벡터표현, 문서의 테이블 표현이 가진 **한 가지 장점은 어떤 정보도 소실되지 않는다는 것.**
    - **각 col이 어떤 단어에 대응되는지에 대한 정보만 유지하면 원핫 벡터 테이블을 통해 元 문서 복원이 가능.**
    - 이런 장점때문에 NN 같은데서 원핫 단어 벡터를 많이 사용함. 물론 메모리와 trade-off 해야하지만...
- 원핫 벡터 테이블을 **피아노 롤** or **음악상자 원통** 같은 개념으로 생각할 수 있음.
- 궁극적으로 하고자 하는것은 원핫 벡터들을 이용해 새로운 문장을 만드는것. LSTM을 이용한 NN 같은 모형을 사용해 만들어낼 수 있음. 추후 chapter 9, 10. 에서.

- 원핫 벡터 테이블은 컴퓨터가 좋아하는 **이진수**. 또한 **원문의 모든 세부 사항을 유지**하고 있음.
- 하지만 **문장이 짧음에도 불구하고 테이블이 상당히 큼**. $\rightarrow$ **차원축소** 과정이 필요.

- 우리가 원하는 것은 **문서의 의미를 압축해 그 본질을 추출**하는 것.
    - **완벽한 복원 능력을 포기**하고 문서 하나를 커다란 테이블이 아닌 **벡터 하나로 압축**하자.
- 이때 사용하는 것이 **단어 빈도(frequency) 벡터**. or **단어 모음 벡터**.
    - 원핫벡터와는 다르게 **단어 순서에 관한 정보가 없음**. 단어의 위치가 아닌 **출현 횟수**만을 담고 있음.
    - 문서를 재생하지 못하는 대신, **하나의 문서 or 문장 전체를 단 하나의 벡터로 표현**할 수 있음.
- 단어 모음 벡터는 피아노 건반 전체를 두드리는 것이 아닌, **듣기 좋은 화음을 연주**하는 셈.
    - **서로 잘 어울리는 음(의미상 연관이 있는 단어)들의 집합을 표현**.

- 특정 단어의 존재 여부를 알려주는 이진 벡터를 구축해보자.
    - 이런 이진 벡터는 어떤 단어가 어떤 문서에 쓰였는지 말해주는 문서 검색 indexing에 유용.
    - 특정 단어가 어디에 있는지 알려주는 것이 아닌, 어떤 문장 or 문서에서 나왔는지를 말해주는 것.
- 앞서 생성한 sentence를 이진 단어 모음 벡터로 압축해보자.

In [7]:
sentence_bow = {}

# sentence를 공백기준으로 나눠서
for token in sentence.split():
    # 해당 단어가 한번 출현했으면 Dict에 추가.
    # 이진 단어모음 벡터 (이진 희소 벡터)가 Dict에 저장되어 메모리 낭비가 그리 심하지 않음.
    sentence_bow[token] = 1

sorted(sentence_bow.items())

[('23.', 1),
 ('Charlie', 1),
 ('Data', 1),
 ('Science', 1),
 ('age', 1),
 ('at', 1),
 ('of', 1),
 ('started', 1),
 ('study', 1),
 ('the', 1)]

- 이진 벡터가 Dict 자료형에 저장되어서, 희소행렬 보다 저장공간이 훨씬 절약됨.
    - 값이 1인 항목만을 저장.
    - 따라서 어휘가 수천개~수만개 라도 문서에 출현한 단어에 대해서만 저장공간이 소비됨.
- 이 Dict의 **각 항목에 해당 단어가 어휘의 몇 번째 단어인지 말해주는 index값을 넣어준다**면 저장공간을 더 줄일 수 있음.
- 이를 위한 자료형이 바로 **pd.Series**.
    - Series를 DataFrame으로 감싸면, 이진 벡터 말뭉치(corpus)에 더 많은 문장 추가가 가능.

In [8]:
import pandas as pd

# 1. List comprehension으로 items를 생성한 후
# 2. 이를 dict 자료형으로 변환,
# 3. 이를 다시 pd.Series로 변환한 후,
# 4. 임의의 sent를 column으로 하는 pd.DataFrame을 생성. (.T를 통해 Transpose해서 보기 좋게)
df = pd.DataFrame(pd.Series(dict([(token, 1) for token in sentence.split()])), columns=['sent']).T
df

,Charlie,started,Data,Science,study,at,the,age,of,23.
sent,1,1,1,1,1,1,1,1,1,1


In [9]:
[(token, 1) for token in sentence.split()] # list comprehension으로 items 생성

[('Charlie', 1),
 ('started', 1),
 ('Data', 1),
 ('Science', 1),
 ('study', 1),
 ('at', 1),
 ('the', 1),
 ('age', 1),
 ('of', 1),
 ('23.', 1)]

문장 몇개를 더 추가해 앞서 정의한 df가 어떻게 성장하는지 살펴보자.  <br>

In [15]:
sentences = """Charlie started Data Science study at the age of 23.\n"""
sentences += """Study was done because of graduation project.\n"""
sentences += """He graduated Korea Aerospace University at 2021.\n"""
sentences += """And he started studying Data Science again and again."""
sentences

'Charlie started Data Science study at the age of 23.\nStudy was done because of graduation project.\nHe graduated Korea Aerospace University at 2021.\nAnd he started studying Data Science again and again.'

In [28]:
corpus = {}

# 개행문자 \n를 기준으로
# 각 문장별로 토큰 분리, 분리한 토큰을 corpus[sentence (0~3)] 에 넣는다. 말뭉치에 넣기.
for i, sent in enumerate(sentences.split('\n')):
    corpus[f'sentence{i}'] = dict((tok, 1) for tok in sent.split())
corpus

{'sentence0': {'Charlie': 1,
  'started': 1,
  'Data': 1,
  'Science': 1,
  'study': 1,
  'at': 1,
  'the': 1,
  'age': 1,
  'of': 1,
  '23.': 1},
 'sentence1': {'Study': 1,
  'was': 1,
  'done': 1,
  'because': 1,
  'of': 1,
  'graduation': 1,
  'project.': 1},
 'sentence2': {'He': 1,
  'graduated': 1,
  'Korea': 1,
  'Aerospace': 1,
  'University': 1,
  'at': 1,
  '2021.': 1},
 'sentence3': {'And': 1,
  'he': 1,
  'started': 1,
  'studying': 1,
  'Data': 1,
  'Science': 1,
  'again': 1,
  'and': 1,
  'again.': 1}}

In [29]:
# 앞서 생성한 말뭉치(corpus)를 DataFrame으로 변경
# fillna를 통해 다른 문장에 없는 토큰은 0으로 대체.
df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
df

,Charlie,started,Data,Science,study,at,the,age,of,23.,...,Korea,Aerospace,University,2021.,And,he,studying,again,and,again.
sentence0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
sentence1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
sentence2,0,0,0,0,0,1,0,0,0,0,...,1,1,1,1,0,0,0,0,0,0
sentence3,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1


### Inner Product

<br>

- inner product는 NLP에서 아주 자주 쓰이므로 개념을 확실이 이해하자.
- **내적(inner product)** : 두 연산 대상(벡터 또는 2차원 행렬)의 '안쪽' 차원이 같아야 한다는 제약 때문에 생긴 연산.
    - **벡터의 경우 두 벡터의 성분 개수가 같아야** 함.
    - **행렬의 경우 왼쪽 행렬의 col 수와 오른쪽 행렬의 row 수가 같아야** 함.
        - **INNER JOIN 연산**으로 생각하면 됨.
    - **결과는 하나의 스칼라 값.** 그래서 **스칼라 곱(scala product)**이라고도 함.
    - 연산이 dot ($\cdot$) 으로 이뤄지기 때문에 **dot product** 라고도 함.
    - 연산? **같은 위치에 있는 두 성분을 각각 곱하고, 그 결과들을 모두 더해서 나온 스칼라 값이 결과값.**
        - 행렬 내적은 matrix product. **np.matmul** 이나 **@ 연산자**로 계산할 수 있음.
- 내적과 대비되는 연산으로 **outer product (cross product)**가 있음. 연산자가 $\times$ 라서.
    - 결과는 벡터.
- 예시를 보자.

In [25]:
v1 = np.array([1,2,3])
v2 = np.array([2,3,4])
print(v1, v2)

v1.dot(v2) # 단순 내적.

(v1 * v2).sum() # 마찬가지로 단순 내적. 같은 결과.

sum([x1 * x2 for x1, x2 in zip(v1, v2)]) # 같은 결과긴 하지만 매우 느림. 쓰지말자.

[1 2 3] [2 3 4]


20

### 두 단어 모음의 중복 측정

<br>

- **두 단어 모음 벡터가 얼마나 겹치는지 측정**한다면, 해당 **문장들이 단어들을 얼마나 비슷하게 사용하는지 알 수 있음.**
- 이렇게 측정한 측도는 **두 문장의 의미가 얼마나 비슷한지를 어느정도 말**해주게 되는 셈.
- 내적을 통해 원래 예제 문장(sentence0 ~ sentence3)과 몇가지 새 문장의 단어 모음 벡터 중복도를 측정해보자.

In [35]:
df = df.T # 다시 원래형태로 Transpose
df

,sentence0,sentence1,sentence2,sentence3
Charlie,1,0,0,0
started,1,0,0,1
Data,1,0,0,1
Science,1,0,0,1
study,1,0,0,0
at,1,0,1,0
the,1,0,0,0
age,1,0,0,0
of,1,1,0,0
23.,1,0,0,0


In [36]:
print(df['sentence0'].dot(df['sentence1']))
print(df['sentence0'].dot(df['sentence2']))
print(df.sentence0.dot(df.sentence3))

1
1
3


    결과는 두 문장 sentence0-sentence1~3이 각각 1,1,3개의 단어를 공통으로 사용함을 말해줌.

- 이런 단어 중복도는 두 문장의 **유사성**에 관한 측정도.
- 해당 내적 값이 1 이상이 되게 한 단어는 뭔지 한번 찾아보자.

In [39]:
print([(key, value) for (key, value) in (df['sentence0'] & df.sentence1).items() if value])
print([(key, value) for (key, value) in (df['sentence0'] & df.sentence2).items() if value])
print([(key, value) for (key, value) in (df['sentence0'] & df.sentence3).items() if value])

[('of', 1)]
[('at', 1)]
[('started', 1), ('Data', 1), ('Science', 1)]


    이것이 바로 벡터 공간 모형. Vector Space Model. VSM.

- 이런 VSM의 단어 모음 벡터들에 대해 내적 연산 뿐만 아니라 사칙연산, 논리연산을 적용할 수 있고, 거리 or 각도 계산도 할 수 있음.
- 문서를 이처럼 이진 벡터로 표현하는 것은 아주 강력함. 이게 문서 조회 및 검색의 주된 수단이었음.

### 토큰 개선

<br>

- 어떤 경우엔 단어들을 공백이 아닌 **다른 문자로 구분**하는 것이 나을 수도 있음.
    - 앞서 생성한 토큰 생성기는 26. 을 하나의 토큰으로 간주했음.
    - 이런 문제를 해결하기 위해선 **공백 문자 뿐만이 아닌 마침표, 쉼표, 따옴표 같은 부호들도 구분자로 써야함.**
    - 독립 토큰으로 다뤄야 할 때도 있고, 아예 제거해야 할 때도 있으니 적절하게.
- 앞서 나온 26. 토큰은 마침표가 붙여져서 바람직하지 않은 토큰이 되었음.
- 이런 형태는 **NLP 파이프라인 이후 단계들을 혼란에 빠뜨릴 위험이 크다.**
- 후행 마침표를 제거하고 토큰을 뽑는 방법을 살펴보자.

In [40]:
import re

sentence = 'Charlie started Data Science study at the age of 23.'

# 정규표현식 이용.
# 주어진 문장을 공백(\s)이나 몇가지 문장부호(.,;!?)를 기준으로 분할.
tokens = re.split(r'[-\s.,;!?]+', sentence)
tokens

['Charlie',
 'started',
 'Data',
 'Science',
 'study',
 'at',
 'the',
 'age',
 'of',
 '23',
 '']

    이용한 정규표현식 : [-\s.,;!?]+
    
- 대괄호 **[ ]** : 주어진 텍스트가 부합해야 할 문자들의 집합 정의. character class (문자 부류) 라 부름.
- 대괄호 옆 **+** : 주어진 문자 부류의 문자들이 **하나 이상 부합해야 함(한번이라도 나와야 한다)**을 의미.
- **\s** : 다양한 공백문자들을 대표. **[\s] == [  \t\n\r\f\v]** 와 같은 의미.
- 대괄호 안 단일 **-** : 범위 지정이 아닌 **단일 문자 지정**.
    - **왼쪽 대괄호 바로 옆에 지정**하거나, **앞에 \ 를 붙여서** 써야 함.

<br>

- 대괄호 안에 **문자 범위**를 지정할 수도 있다.
    - **[0-9]** : 0부터 9까지의 십진 숫자.
    - **[_a-zA-Z]** : 밑줄문자, 모든 영어 대소문자 부합.
- **소괄호** : 정규 표현식의 **한 부분을 하나의 그룹으로 지정**하는 용도로 사용.
    - 소괄호 그룹은 **전체가 문자열의 일부와 부합**해야 함.
        - 소괄호 쌍 안의 표현식 전체가 현재 위치의 문자들과 부합해야 소괄호 쌍 다음의 패턴으로 넘어감.
        
<br>
    
- **re.split** : **두번째 인수로 문자열**을 받아, **해당 문자열을 훑으면서 정규 표현식에 지정된 패턴과 부합하는 부분 문자열을 찾음.**
    - 부합하는 부분을 발견하면, 부합한 마지막 문자 이전 부분을 결과 집합에 추가, 이 과정을 반복.
    - str.split() 처럼 동작하지만, 단순한 구분자가 아닌 정규 표현식을 구분 기준으로 적용.

#### 단어 분리를 위한 정규 표현식 개선 - re.compile

<br>

- 정규표현식을 한번 컴파일하면 여러곳에서 재사용이 가능.
- re.compile() 을 통해 표현식을 미리 컴파일, 결과를 하나의 인수로서 사용하면 된다.
- 코드 실행속도가 눈에띄게 빨라지는 건 아니지만, 표현식이 많아지는 경우 속도의 이득이 생길 수 있음.

In [41]:
pattern = re.compile(r'([-\s.,;!?])+')
tokens = pattern.split(sentence)
tokens

['Charlie',
 ' ',
 'started',
 ' ',
 'Data',
 ' ',
 'Science',
 ' ',
 'study',
 ' ',
 'at',
 ' ',
 'the',
 ' ',
 'age',
 ' ',
 'of',
 ' ',
 '23',
 '.',
 '']

    마침표 분리는 성공적인데, 불필요한 공백과 문자부호가 토큰이 되어버림.
    한번 걸러보자.

In [42]:
[x for x in tokens if x and x not in '- \t\n.,;!?'] # 공백문자 (whitespace) 추가

['Charlie',
 'started',
 'Data',
 'Science',
 'study',
 'at',
 'the',
 'age',
 'of',
 '23']

In [44]:
list(filter(lambda x: x if x and x not in '- \t\n.,;!?' else None, tokens)) # 얘도 동일한 결과.

['Charlie',
 'started',
 'Data',
 'Science',
 'study',
 'at',
 'the',
 'age',
 'of',
 '23']

    re 도 좋지만, 이보다 더 좋은 regex를 쓰자. re랑 완전호환 됨.

- 이렇게 쓰는 토큰 생성기는 예외사항이 생길수록 금방 복잡해짐.
- 마침표를 구분자로 쓰면 문장들은 잘 구분하지만, 숫자의 소수점을 이상하게 분리해버리는 결과가 나올 수 있음.
- 또는, 이모티콘에 마침표가 들어가는 경우도 있음. ^.^ 이런거. 찢어버리면 안되용...

<br>

- 이를 위해 토큰화 기능을 구현한 라이브러리가 여러개 있음. 대표적인 것만 살펴보면 다음과 같다.
    - spaCy : 정확함, 유연함, 빠름, Python
    - CoreNLP : 더 정확하지만 덜 유연함. 빠름, Java 8에 의존.
    - NLTK : 여러곳에서 쓰이는 표준 라이브러리, 유명함, Python
- NLTK를 써서 앞서 썻던 문장을 토큰화 해보자.

In [47]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+|$[0-9.]+|\S+')
tokenizer.tokenize(sentence)

['Charlie',
 'started',
 'Data',
 'Science',
 'study',
 'at',
 'the',
 'age',
 'of',
 '23',
 '.']

- 앞서 만들은 정규 표현식보다 좀 더 유연하게 잘 토큰화 했다.
- NLTK 패키지엔 TreebankWordTokenizer라는 함수도 있음. RegexpTokenizer보다 강력함.
    - Penn Treebank tokenization에 기초한 함수. 영어 단어 토큰화에 쓰이는 다양한 규칙을 담고있음.
        - 문장 끝 부호를 인접 토큰들과 분리하면서도 소수점 있는 수치는 하나의 토큰으로 유지.
        - 축약형 단어들을 위한 규칙도 갖추고 있음. wasn't $\rightarrow$ was, n't 로 토큰화. 
- 예시를 한번 보자.

In [52]:
from nltk.tokenize import TreebankWordTokenizer

sentence = "Monticello wasn't designated as UNESCO World Heritage Site until 1987."
tokenizer = TreebankWordTokenizer()
tokenizer.tokenize(sentence)

['Monticello',
 'was',
 "n't",
 'designated',
 'as',
 'UNESCO',
 'World',
 'Heritage',
 'Site',
 'until',
 '1987',
 '.']

#### 축약형

<br>

- wasn't 를 was와 n't로 분리하는 것이 바람직함. 왜그럴까?
    - 구문 트리 파서가 입력을 미리 정해진 규칙들에 기초해 일관된 토큰들로 다루기 위해선 was와 not으로 분리해야 함.
    - 축약형 단어를 구 구성 단어들로 분리하면 모든 가능한 축약형 단어를 예측할 필요 없이 개별 단어의 철자 변형들만 고민하면 됨.
    - 그래서 의존성 트리 파서나 구문 파서를 만들기 쉬워짐. 이런 이유 때문. 음...

    # 이런거도 있답니다. casual_tokenize
    # SNS에서 흔히 볼수 있는 비형식적이고 이모지가 난무하는 텍스트를 다루기 위한 NLTK의 패키지.
    # 텍스트에서 사용자 이름을 제거하고, 토큰 안에서 반복되는 문자들을 줄여줌.

### N-gram을 이용한 어휘 확장

<br>

- 다음과 같은 문장을 생각해보자.

        I scream, you scream, we all scream for ice cream.

- 너와 나 우리 모두가 바라는 것은 ice cream. ice나 cream이 아님. 하나의 단위로 저장되게 하는 방법이 필요.

#### 해결책은 바로 N-gram을 사용하는 것.

<br>

- *n-gram*은 **요소들의 순차열(보통 문자열)에서 추출한 최대 n개의 요소로 이뤄진 순차열.**
    - ice cream은 2-gram이 되는 셈.
    - Johann Sebastian Bach 는 3-gram이 되는 셈.
- n-gram 단어들이 **반드시 어떤 특별한 의미를 가지는 것은 아님.**
    - 복합어가 아닌 단어 조합도 얼마든지 n-gram이 될 수 있음. 
    - n-gram은 단순히 **일련의 요소 중 인접한 몇개의 요소를 묶은 것.**
    
<br>

- n-gram이 왜 필요할까?
    - **토큰들을 하나의 단어모음 벡터로 표현하게 되면 단어 순서에 담긴 정보가 사라지게 됨.**
    - 하나로 된 토큰이라는 개념을 여러 단어로 이뤄진 토큰인 n-gram으로 확장하면 파이프라인은 **문장 단어 순서에 담겨진 의미를 좀 더 많이 유지**할 수 있음.
        - 부정을 뜻하는 not은 다른 단어와 붙어있을 때 그 의미가 뚜렷해 짐.
        - n-gram tokenization을 하지 않으면 not은 그냥 홀로 떨어진 단어.
        - 2-gram인 'was not'은 각자 개별로 있을때보다 더 많은 의미를 내포하고 있음.
        - 즉, 파이프라인에서 **인접 단어들을 묶으면 각 단어의 문맥(context)이 어느정도 형성됨.**

<br>

- 예시를 한번 보자.

In [56]:
# 기존의 1-그램 토큰 생성기.
sentence = 'Charlie started Data Science study at the age of 23.'
pattern = re.compile(r'([-\s.,;!?])+')
tokens = [x for x in tokens if x and x not in '- \t\n.,;!?']
tokens

['Charlie',
 'started',
 'Data',
 'Science',
 'study',
 'at',
 'the',
 'age',
 'of',
 '23']

In [58]:
# nltk의 n-그램 토큰화 함수 사용
from nltk.util import ngrams
list(ngrams(tokens, 2)), list(ngrams(tokens, 3)) # 그냥 단순히 토큰과 n을 인자로 넣어주면 된다!

([('Charlie', 'started'),
  ('started', 'Data'),
  ('Data', 'Science'),
  ('Science', 'study'),
  ('study', 'at'),
  ('at', 'the'),
  ('the', 'age'),
  ('age', 'of'),
  ('of', '23')],
 [('Charlie', 'started', 'Data'),
  ('started', 'Data', 'Science'),
  ('Data', 'Science', 'study'),
  ('Science', 'study', 'at'),
  ('study', 'at', 'the'),
  ('at', 'the', 'age'),
  ('the', 'age', 'of'),
  ('age', 'of', '23')])

    # 참고
            ngrams 함수는 파이썬의 generator 객체를 return함.
            generator는 iterator처럼 작동하는 함수. 
            모든 n-gram의 순차열을 한번에 돌려주지 않고,
            한번에 하나씩만 yield(산출) 함.
            
            이러면 뭐가 좋을까? for loop에서 유용함.
            generator를 통해 순차열의 모든 요소를 메모리에 담지 않고, 
            개별 하나씩만 가져와 처리가 가능.
            
            return된 n-그램들을 한번에 조사하고 싶다면 산출 결과를 list로 만들면 됨.
            (But 큰 문서에 대한 실제 토큰화 작업에는 시간이 오래걸림. 바람직하지 않다.)

- 산출한 각 n-그램은 하나의 tuple.
- 한 n-그램의 모든 토큰을 연결해 하나의 문자열로 만드는 것 또한 가능.
- 이렇게 만들면 파이프라인 이후 단계들이 일관되게 문자열을 입력으로 받게 되므로 설계가 단순해짐.

In [59]:
two_grams = list(ngrams(tokens, 2))
[' '.join(x) for x in two_grams]

['Charlie started',
 'started Data',
 'Data Science',
 'Science study',
 'study at',
 'at the',
 'the age',
 'age of',
 'of 23']

- 문제점이 있다.
    - Data Sciene 같은 2-gram은 많은 곳에서 등장하겠지만,
    - of 23, started Data 같은 것이 출현하는 문서는 극히 드물 것.
- 어떤 토큰이나 n-gram이 **극히 드물게만 나타난다**는 의미는, **해당 토큰이 다른 단어들과 상관관계가 거의 없다**는 의미.
- 즉, **희소한 n-gram은 분류 문제에 큰 도움이 되지 않는다.**
    - 대부분의 2-gram은 상당히 드물고, 3-gram과 4-gram은 더 드물다.

<br>

- 단어 조합이 개별 단어보다 훨씬 sparse하므로, 어휘 크기는 말뭉치의 모든 문서에 있는 n-gram들의 수에 지수적으로 접근한다.
    - 특징 벡터의 차원 수가 문서의 길이보다 크면 특징 추출 단계가 오히려 파이프라인 생산성에 해를 미칠 수 있음
    - 그런 경우 ML 모형이 벡터들에 overfitting하는 결과를 피하기가 거의 불가능.
        - chapter 3.에서 문서 빈도 통계량 등을 이용해 ML에서 유용하지 않은 sparse n-gram을 골라내는 방법을 논의함.
- at the 같은 경우는 드문 조합이 아니지만, **너무 많은 문서에 등장함**. **문서 의미 구분에 큰 도움이 X**.
- **너무 자주나오는 토큰이나 n-gram은 무시**하는 것이 좋음. **불용어 필터링.**

#### Stop-word (불용어)

<br>

- **아주 자주 출현하지만, 문구 의미에 관한 실질적 의미는 별로 담고 있지 않는 단어.**
    - a, an, the, this 이런거.
- 불용어 자체는 정보를 별로 전달안하지만, **n-그램에 포함되면 중요환 관계 정보를 제공할 수 있음.**
    - Mark reported to the CEO
    - 4-그램 단위로 보면 reported to the CEO가 만들어짐.
    - 불용어 to, the를 지우면 reported CEO가 되서 관계 구조 정보가 사라짐.
    - 하지만 **불용어를 사용하려면 n-그램의 n이 길어지는 단점**이 있음.
- 불용어 필터의 구체적 **설계는 프로그램 마다 다름**. 불용어는 전체 어휘의 극히 일부분만 차지. 

<br>

- 토큰화 과정에서 불용어들을 무조건 제거하기로 했다면, 파이썬의 list comprehension을 써서 간단히 제거 가능.
- 예시를 보자.

In [60]:
stop_words = ['a', 'an', 'the', 'on', 'of', 'off', 'this', 'is']
tokens = ['the', 'house', 'is', 'on', 'fire']
tokens_without_stopwords = [x for x in tokens if x not in stop_words]
tokens_without_stopwords

['house', 'fire']

- 문장에 따라 불용어들이 담은 의미 양이 다를 수 있음. 자명.
- 표준적인 불용어 목록을 보고자 한다면 NLTK에 정의된 목록을 한번 살펴보자.

In [61]:
import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
len(stop_words), stop_words[:10]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\skdbs\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(179,
 ['i',
  'me',
  'my',
  'myself',
  'we',
  'our',
  'ours',
  'ourselves',
  'you',
  "you're"])

- NLP 관점에서, 1인칭 문장들이 계속 나오는 문서는 상당히 지루하고, 정보량이 적은 경우가 많음.
- sklearn의 불용어 집합과 NLTK의 불용어 집합을 함께 섞어서 사용할 수도 있음.

### 어휘 정규화 (Normalization)

<br>

- 어휘의 크기를 줄이는 또 다른 기법.
- **비슷한 토큰들을 하나의 정규화된 형태로 결합**하는 것.
- 어휘의 **토큰 개수가 줄어들고**, **같은 의미지만 철자가 다른 토큰 or n-gram들을 동일한 의미 단위로 취급** 가능.
- **Overfitting 가능성 또한 작아짐**.

#### Case Folding (대소문자 합치기)

<br>

- **대소문자 구성만 다른 단어들을 하나로 통합**하는 것.
    - 왜 필요? **하나의 단어가 위치 or 저자의 의도에 따라 대소문자 구성이 달라지기 때문**.
        - 첫글자를 대문자로 쓰거나, 강조를 위해 모두 대문자로 쓰는 경우들.
- 불규칙한 대소문자 구성들을 하나의 구성으로 정규화 한다는 점에서, **case normalization (대소문자 정규화)**라고도 함.
- 대소문자 구성을 정규화하는 것은 **어휘의 크기를 줄이고 NLP 파이프라인을 일반화** 하는 방법 중 하나.
- 또한, **같은 의미를 가진 여러 단어를 하나의 토큰으로 병합**하는데 도움이 됨.

<br>

- 하지만 대소문자 구성이 나름의 의미를 지닐때도 있음. 
    - **고유 명사** 같은 경우. 
    - **개체명을 인식하는 것이 파이프라인의 주 과제라면, 대소문자 구성은 유지**하는 것이 좋음.
    - 하지만 유지하면 어휘가 약 2배 커지며, 그만큼 메모리와 처리 시간도 2배로 커짐.

<br>

- list comprehension을 통해 대소문자 구성을 쉽게 정규화 할 수 있음.
    - 문서 **전체에서 정규화 하는것이 바람직함이 확실**한 경우, **원문 텍스트 전체에 대해 lower()를 한 후 토큰화를 진행**하면 된다.
- 하지만 이러면 **Camel Case 표기법으로 된 단어들을 쉽게 토큰화 하기가 힘들어짐.**
    - Age를 검색하면 age를 검색할때와는 다른 문서들이 나옴.
    - Age는 New Age 같은 문구에 등장하지만 age는 at the age of 같은 문구에서 등장.
    - 어휘를 정규화 하게 되면, 사용자가 입력한 검색어의 대소문자 구성과는 무관하게 age에 관한 문서만 반환.
- 좀 더 고급지게 한다면 첫단어만 소문자로 만들고, 나머지는 대소문자 구성을 유지하는 것.

In [62]:
tokens = ['House', 'Visitor', 'Center']
norm_tokens = [x.lower() for x in tokens]
norm_tokens

['house', 'visitor', 'center']

    정규화는 재현율을 높여주긴 하지만, 정밀도를 낮춤.
    사용자가 관심 없는 문서들이 검색 결과에 많이 포함됨.
    --> 이런걸 피하고자 구글 검색에 " " 로 감싸서 감색하는 기능같은게 있음.
    대소문자 구성을 정규화한 n그램들과 원래 대소문자 구성을 유지한 n그램으로 색인 유지.

#### Stemming (어간 추출)


<br>

    https://wikidocs.net/21707
- **단어 끝의 복수형 접미사나 소유격 접미사에 의한 차이를 제거**하는 것.
- **한 단어의 여러 변형에 공통으로 존재하는 어간을 식별해 단어들을 정규화 하는 작업**.
    - housing과 houses의 공통 어간은 house.
    - 어간이 반드시 영어 사전에 나오는 정확한 사전일 필요는 X.
    - NLP에서의 어간은 **한 단어의 여러 철자 변형들을 대표하는 하나의 토큰 또는 이름표** 일 뿐.
- 장점? **문서에 담긴 정보와 의미를 최대한 유지하면서 어휘의 크기를 줄임**. 일종의 **차원축소 기법**. **모형 일반화**.
- 키워드 검색, 정보 조회에서 중요. 관련 문서나 웹 페이지가 누락되지 않도록 검색 결과를 확장하는데 도움을 줌.
    - developing houses in Portland 를 검색했을 때, *houses가 있는 문서 뿐만이 아닌 house나 housing이라는 문서도 검색 결과에 포함되는 이유는 **hous** 라는 하나의 공통 어간을 추출해서* 나오는 것.

<br>

- 어간추출 때문에 검색 엔진의 **정밀도가 크게 감소**할 수 있음. 무관한 문서도 나올 수 있다는 것.
    - **False-Positive(거짓양성)비율이 증가**할 수 있음.
    - 이도 마찬가지로 " " 처럼 검색옵션을 통해 해결할 수 있음.

<br>

- 후행 s들을 처리하는 간단한 어간 추출기 예시를 보자.

In [65]:
def stem(phrase):
    return ' '.join([re.findall('^(.*ss|.*?)(s)?$', word)[0][0].strip("'")
                    for word in phrase.lower().split()])
stem('houses'), stem("Doctor House's calls")

('house', 'doctor house call')

^(.*ss|.*?)(s)?&#36; 뭘까 이게?
    
- 만일 단어가 둘 이상의 s로 끝나면, 어간은 그 단어 자체이고 접미사는 빈 문자열.
- 만일 단어가 1개의 s로 끝나면, 어간은 단어에서 s를 제외한 부분이고 접미사는 s.
- 만일 단어가 s로 끝나지 않으면, 어간은 그 단어 자체이고 접미사는 X.

<br>

    하지만 이 함수는 복잡한 경우엔 잘 작동을 못함. housing 같은 경우.
    nltk의 PorterStemmer(Porter Algorithm)를 쓰면 쉽게 추출할 수 있다.

In [66]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
' '.join([stemmer.stem(w).strip("'") for w in "dish washer's washed dishes".split()])

'dish washer wash dish'

#### Lemmatization (표제어 추출)

<br>

- 단어들을 **그 바탕 의미를 담은 어근(root) 수준으로 내려가 정규화** 하는 작업.
- **바탕 의미가 같지만 철자가 다른 여러 단어에 대해 파이프라인이 일관된 반응을 하는데 크게 도움**을 줌.
- 이 또한 **대응할 단어 수를 줄임**. 언어 모형의 **차원 수를 줄이는 차원축소 기법**. **모형 일반화 가능**.
    - 대신 **모형이 좀 덜 정확**해짐. *뿌리가 같지만 의기마 완전히 같지는 않은 철자 변형을 모두 같은 단어로 봐서*.
        - chat, chatter, chatty, chatting, chatbot 을 똑같이 취급해버림.
    - 의도적으로 이렇게 만들 수도 있음. 이를 spoofing 이라고 함.

<br>

- 의미를 고려해 단어를 정규화 하므로 **어간 추출이나 대소문자 정규화보다 좀 더 정확한 정규화 방법**.
- 동의어들과 단어 어미들에 관한 **지식 베이스를 활용**해 **실제 의미가 비슷한 단어들만 하나의 토큰으로 통합**.
- 단어의 철자 뿐만 아닌 **품사(part of speech, POS)도 활용**해 **정밀도를 개선**하기도 함.
    - 이럴땐 **단어가 어떤 역할을 하는지 품사 태그(꼬리표)를 단어에 부여**함. 
        - 예전 예제들 돌려봤을때 단어마다 'v', 'n' 이렇게 붙어있던거.

<br>

- ***품사에 기초한 표제어 추출이 어간 추출보다 더 나은 어근을 식별할 수 있는 이유가 뭘까.***
    - better를 생각해보자.
    - 어간 추출기는 어미 er를 제거해 bett나 bet 같은 어간을 추출할 것.
    - 이렇게 하면 better는 bets나 Bet's 처럼 전혀 다른 의미의 단어들과 묶임.
    - 하지만 표제어 추출기는 better를 betterment, best 처럼 의미상으로 가까운 단어들과 묶음.
- 그래서 대부분의 어플리케이션에선 표제어 추출기보다 어간 추출기보다 나음.
    - 어간 추출기는 대규모 정보 검색 어플리케이션 같은 곳에서나 쓰임.
- **어간 추출 바로 앞 단계에 표제어 추출 단계를 배치**해 더 나은 결과를 얻을 수도 있음.
    - 즉, 어간 추출기 앞에 표제어 추출기를 두게 되면, 어간 추출기만 사용했을 때 보다 **차원이 더 많이 축소되고 정보 검색 재현율도 높아지게 된다.**

<br>

- 표제어 식별 예제를 보자. NLTK의 WordNetLemmatizer라는 추출기가 있음.
    - **두번째 인수**로 **단어의 품사 정보**를 요구함.

In [69]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('better', pos='a'), lemmatizer.lemmatize('goods', pos='a'), lemmatizer.lemmatize('goodness', pos='a'), lemmatizer.lemmatize('goodness', pos='n')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\skdbs\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


('good', 'goods', 'goodness', 'goodness')

    품사를 명시적으로 지정하지 않으면 WordNetLemmatizer는 그 단어가 명사라고 가정함.
    이 추출기는 프린스턴 대학교의 WordNet 단어 의미 그래프에 있는 단어 연결 관계만을 이용.

- 일반적으로 어간 추출이 표제어 추출보다 빠르고, 필요한 코드와 자료집합도 덜 복잡.
- 하지만 어간 추출기는 표제어 추출기보다 실수가 더 잦고, 더 많은 단어를 하나의 토큰으로 합침. 정보손실 증가.